In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Initialising the CNN
classifier = Sequential()

# Step1 - Convolution
classifier.add(Convolution2D(300, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(Dropout(0.2))
# Step2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))


# Adding a second convolution layer
classifier.add(Convolution2D(128, 3, 3, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.2))
# step3 - Flattening
classifier.add(Flatten())


# Step4 - Full COnnection
classifier.add(Dense(units = 64, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

   
# Compiling the CNN
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])


lrd = ReduceLROnPlateau(monitor = 'val_loss',
                         patience = 200,
                         verbose = 1,
                         factor = 0.75,
                         min_lr = 1e-10)
mcp = ModelCheckpoint('model.h5')

es = EarlyStopping(verbose=1, patience=60)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(
        '../input/skin-cencer-dataset-images/train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        '../input/skin-cencer-dataset-images/test',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

In [ ]:
history = classifier.fit_generator(training_set,steps_per_epoch = 10, epochs=60, callbacks=[lrd, mcp, es], validation_data = test_set,validation_steps = 200)

In [ ]:
classifier.summary()

In [ ]:
history.history.keys()

In [ ]:
import pandas as pd
losses = pd.DataFrame(classifier.history.history)

losses[['loss','val_loss']].plot()

In [ ]:
losses = pd.DataFrame(classifier.history.history)

losses[['loss','accuracy']].plot()

In [ ]:
classifier.metrics_names

In [ ]:
classifier.evaluate_generator(test_set)


In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
import numpy as np

test_image = image.load_img('../input/skin-cencer-dataset-images/data/test/benign/10.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'Uninfected'
else:
    prediction = 'Parasitised'
prediction

In [ ]:
test_image = image.load_img('../input/skin-cencer-dataset-images/data/test/benign/1003.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 0:
    prediction = 'Uninfected'
else:
    prediction = 'Parasitised'
prediction